<a href="https://colab.research.google.com/github/Qwerty125-blip/KanHope/blob/main/Notebooks/roberta-mbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers 

In [2]:
from transformers import BertModel
import os
import torch
#from googletrans import Translator
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [3]:
train = pd.read_csv('/content/multichannelhope.csv')
#train['labels']=LabelEncoder().fit_transform(train['class'])  
train=train.drop(columns=['Unnamed: 0'])
train.head(10)

,sentence,labels,translation
0,YouTube subscriber at list lack kooda Ella yel...,1,YouTube Subscriber at Least Luck Reach Everywhere
1,Thu nem yogyathe ge maha vishnu hesaru ettu en...,1,Thu Name is worthy of the name of Maha Vishnu
2,Super togare teppa,1,Super sharp
3,Super song,1,Super song
4,Lyrics Artha adoru Kay kaal aadsrappa,1,Lyrics mean adoru gay foot dance
5,Anna e game yavudu,1,Anna What's this game
6,Nanu weaks ninda huccha agbitidini boss song ...,0,I am crazy from Weeks to the Boss Song
7,ನಮ್ಮ ಕನ್ನಡ ಇಂಡಸ್ಟ್ರಿ ಯಾಕೆ ನಮ್ಮ ಕಲಾವಿದರನ್ನ ಕೈ ಬ...,0,Why our Kannada Industry ..
8,Ninyavaga hindi li nodidiyappa?? Swalpa link s...,1,When did you see Hindi li ?? Share a little li...
9,ಚಂದನ್ ಶೆಟ್ಟಿ ಟ್ರೊಲ್ ವಿಡಿಯೋ ನೀವು ನೋಡಿ ನೆಗಡ್ಡೆ ಇ...,1,Chandan Shetty Troll Video You See Negadre Idr...


In [4]:
test = pd.read_csv('/content/multichannelhope_test.csv')
#test['lables'] = LabelEncoder().fit_transform(test['class'])
test['labels'] = test['lables']
test = test.drop(columns=['Unnamed: 0', 'lables'])
test.head(10)

,sentence,translation,labels
0,Ban tik tok,Ban tik tok,1
1,Avara desha dalli erodu aethu....nnHagadre Nam...,Avara desha dalli erodu aethu....nnHagadre Nam...,1
2,Game name bro,Game name bro,1
3,ದಿಯಾ ಚಲನಚಿತ್ರ ತುಂಬಾನೇ ಚೆನ್ನಾಗಿದೆ. ಸ್ವಲ್ಪ ಪ್ರಚಾ...,ದಿಯಾ ಚಲನಚಿತ್ರ ತುಂಬಾನೇ ಚೆನ್ನಾಗಿದೆ. ಸ್ವಲ್ಪ ಪ್ರಚಾ...,0
4,Nija nija nam manegu bandidda,Nija nija nam manegu bandidda,1
5,Unlike ಮಾಡಿದವರು ರಶ್ಮಿಕಾ ಅಭಿಮಾನಿಗಳು....,Unlike ಮಾಡಿದವರು ರಶ್ಮಿಕಾ ಅಭಿಮಾನಿಗಳು....,1
6,ಗುರು ಎಲ್ಲಿದ್ದೆ ಗುರು ಇಷ್ಟು ದಿನ rolling_on_the_f...,ಗುರು ಎಲ್ಲಿದ್ದೆ ಗುರು ಇಷ್ಟು ದಿನ rolling_on_the_f...,0
7,Supr bro u r video,Supr bro u r video,1
8,Nanu film nodade ne edict aagiddene,Nanu film nodade ne edict aagiddene,1
9,Bere level aagi ede..appata kannada haadu...,Bere level aagi ede..appata kannada haadu...,0


In [5]:
class cfg:
    debug = False
    max_len = 64
    batch_size = 64
    bert = 'roberta-base'
    mbert = 'bert-base-multilingual-cased'
    lr = 2e-5
    epochs = 10
    save_path = '/content/Saved_Models'

In [6]:
def epoch_time(start_time,end_time):
	elapsed_time = end_time - start_time
	elapsed_mins = int(elapsed_time/60)
	elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
	return elapsed_mins,elapsed_secs

In [7]:
import pandas as pd
from torch.utils.data import Dataset,DataLoader

class RFDataset(Dataset):
  def __init__(self,text, translation, label,tokenizer1, tokenizer2, max_len):
    self.text = text
    self.translation = translation
    self.label = label
    self.tokenizer1 = tokenizer1
    self.tokenizer2 = tokenizer2
    self.max_len = max_len
  
  def __len__(self):
    return len(self.text)
  
  def __getitem__(self,item):
    text = str(self.text[item])
    translation = str(self.translation[item])
    label = self.label[item]

    encoding1 = self.tokenizer1.encode_plus(
        text,
        add_special_tokens=True,
        max_length = self.max_len,
        return_token_type_ids = False,
        padding = 'max_length',
        return_attention_mask= True,
        return_tensors='pt',
        truncation=True
    )
    
    encoding2 = self.tokenizer2.encode_plus(
        translation,
        add_special_tokens=True,
        max_length = self.max_len,
        return_token_type_ids = False,
        padding = 'max_length',
        return_attention_mask= True,
        return_tensors='pt',
        truncation=True
    )
    return {
        'text' : text,
        'translation': translation,
        'input_ids1' : encoding1['input_ids'].flatten(),
        'input_ids2' : encoding2['input_ids'].flatten(),
        'attention_mask1' : encoding1['attention_mask'].flatten(),
        'attention_mask2' : encoding2['attention_mask'].flatten(),
        #'label' : torch.tensor(label,dtype=torch.float)
        'label' : torch.tensor(label, dtype=torch.float)

    }

In [8]:
def create_data_loader(df, tokenizer1, tokenizer2, max_len, batch_size, shuffle=True):
    
    ds =  RFDataset(
        text = df.sentence.to_numpy(),
        translation = df.translation.to_numpy(),
        label = df.labels.to_numpy(),
        tokenizer1 = tokenizer1,
        tokenizer2 = tokenizer2,
        max_len = max_len 
    )
    
    return DataLoader(ds,
                      batch_size = batch_size,
                      shuffle=shuffle,
                      num_workers = 2)

In [9]:
from transformers import AutoTokenizer, AutoModel
tokenizer1 = AutoTokenizer.from_pretrained(cfg.mbert, return_dict=False)
tokenizer2 = AutoTokenizer.from_pretrained(cfg.bert, return_dict=False)

In [10]:
train_data_loader = create_data_loader(train, tokenizer1,tokenizer2, cfg.max_len, cfg.batch_size, shuffle=False)
test_data_loader = create_data_loader(test, tokenizer1, tokenizer2, cfg.max_len, cfg.batch_size, shuffle=False)

In [11]:

import torch.nn as nn
class LinearWeightedAvg(nn.Module):
    def __init__(self, n_inputs):
        super(LinearWeightedAvg, self).__init__()
        self.weights = nn.ParameterList([nn.Parameter(torch.randn(1)) for i in range(n_inputs)])

    def forward(self, input):
        res = 0
        for emb_idx, emb in enumerate(input):
            res += emb * self.weights[emb_idx]
        return res


In [12]:
import torch
import torch.nn as nn
from transformers import AutoModel
class MultichannelBERT(nn.Module):

  def __init__(self, n_classes):
    super(MultichannelBERT, self).__init__()
    self.bert = AutoModel.from_pretrained('roberta-base', return_dict=False)
    self.mbert = AutoModel.from_pretrained('bert-base-multilingual-cased', return_dict = False)
    self.fc1 = nn.Linear(self.bert.config.hidden_size, 128)
    self.fc2 = nn.Linear(self.mbert.config.hidden_size,128)
    self.relu = nn.ReLU()
    self.fc3 = nn.Linear(256,n_classes)
    self.drop = nn.Dropout(p=0.2)
    #self.fc4 = nn.Linear(128, 1)

  def forward(self,input_ids1, attention_mask1, input_ids2, attention_mask2):

    _, mbert_output = self.mbert(
      input_ids = input_ids1,
      attention_mask = attention_mask1
    )

    _, bert_output = self.bert(
        input_ids = input_ids2,
        attention_mask = attention_mask2
    )
    
    bert_out = self.fc1(bert_output)
    mbert_out = self.fc2(mbert_output)

    #Merges both the outputs 
    merged = torch.cat((bert_out, mbert_out),1) 
    act = self.relu(merged)
    out = self.drop(act)
    return self.fc3(out) #self.fc4(bert_out), self.fc4(mbert_output)

In [ ]:
model = MultichannelBERT(1)
model.to(device)

In [14]:
def epoch_time(start_time,end_time):
	elapsed_time = end_time - start_time
	elapsed_mins = int(elapsed_time/60)
	elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
	return elapsed_mins,elapsed_secs

In [15]:
def train_epoch(model,data_loader,loss_fn,optimizer,device,scheduler,n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0

    for data in data_loader:
        input_ids1 = data['input_ids1'].to(device)
        attention_mask1 = data['attention_mask1'].to(device)
        input_ids2 = data['input_ids2'].to(device)
        attention_mask2 = data['attention_mask2'].to(device)
        labels = data['label'].to(device)
        labelsviewed = labels.view(labels.shape[0],1)
         

        outputs = model(
            input_ids1=input_ids1,
            attention_mask1=attention_mask1,
            input_ids2 = input_ids2,
            attention_mask2 =attention_mask2
            )
        _, preds = torch.max(outputs, dim=1)
        preds = [0 if x < 0.5 else 1 for x in outputs]
        preds = torch.tensor(preds).to(device)
        loss = loss_fn(outputs,labelsviewed)
        #loss = loss_fn(outputs, labels)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        torch.cuda.empty_cache()
    return correct_predictions.double() / n_examples, np.mean(losses)

In [16]:
def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0
  with torch.no_grad():
    for data in data_loader:
        input_ids1 = data['input_ids1'].to(device)
        attention_mask1 = data['attention_mask1'].to(device)
        input_ids2 = data['input_ids2'].to(device)
        attention_mask2 = data['attention_mask2'].to(device)
        labels = data['label'].to(device)
        labelsviewed = labels.view(labels.shape[0],1)
 
        outputs = model(
            input_ids1=input_ids1,
            attention_mask1=attention_mask1,
            input_ids2 = input_ids2,
            attention_mask2 = attention_mask2
        )

        #_, preds = torch.max(outputs, dim=1)
        preds = [0 if x < 0.5 else 1 for x in outputs]
        preds = torch.tensor(preds).to(device)
        loss = loss_fn(outputs, labelsviewed)
        #loss = loss_fn(outputs, labels) 
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())
        torch.cuda.empty_cache()
  return correct_predictions.double() / n_examples, np.mean(losses)

In [22]:
EPOCHS = 10
import numpy as np

from transformers import AdamW, get_linear_schedule_with_warmup
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import time
optimizer = AdamW(model.parameters(), lr=cfg.lr, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

#loss = nn.BCELoss().to(device)
loss = nn.BCEWithLogitsLoss().to(device)
#loss = nn.CrossEntropyLoss().to(device)

In [23]:
from collections import defaultdict

history = defaultdict(list)
best_accuracy = 0
for epoch in range(EPOCHS):
 
 
  start_time = time.time()
  train_acc,train_loss = train_epoch(
      model,
      train_data_loader,
      loss,
      optimizer,
      device,
      scheduler,
      len(train)
  )
   
  
  val_acc,val_loss = eval_model(
      model,
      test_data_loader,
      loss,
      device,
      len(test)
  )
  
  end_time = time.time()
  epoch_mins, epoch_secs = epoch_time(start_time, end_time)
  print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
  print(f'Train Loss {train_loss} accuracy {train_acc}')
  print(f'Val Loss {val_loss} accuracy {val_acc}')
  print()

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)

Epoch: 01 | Epoch Time: 1m 28s
Train Loss 0.46458914567684306 accuracy 0.7894926232457719
Val Loss 0.5496874064207077 accuracy 0.7362459546925567

Epoch: 02 | Epoch Time: 1m 28s
Train Loss 0.3987639784470372 accuracy 0.8409499820079165
Val Loss 0.5765828132629395 accuracy 0.7103559870550162

Epoch: 03 | Epoch Time: 1m 28s
Train Loss 0.3231043389131283 accuracy 0.8751349406261244
Val Loss 0.5890236228704453 accuracy 0.7135922330097088

Epoch: 04 | Epoch Time: 1m 28s
Train Loss 0.26379170791171064 accuracy 0.9077006117308384
Val Loss 0.6594173014163971 accuracy 0.7394822006472492

Epoch: 05 | Epoch Time: 1m 28s
Train Loss 0.2310358663571292 accuracy 0.9247930910399423
Val Loss 0.7006612598896027 accuracy 0.7508090614886732

Epoch: 06 | Epoch Time: 1m 28s
Train Loss 0.1946725725870708 accuracy 0.9379273119827275
Val Loss 0.7397001087665558 accuracy 0.7540453074433657

Epoch: 07 | Epoch Time: 1m 28s
Train Loss 0.17535368815578264 accuracy 0.9427851745232098
Val Loss 0.8020942628383636 accu

In [19]:
def get_predictions(model, data_loader):
  model = model.eval()
  sentence = []
  predictions = []
  prediction_probs = []
  real_values = []
  with torch.no_grad():
    for data in data_loader:
      texts = data['text'] 
      input_ids1 = data['input_ids1'].to(device)
      attention_mask1 = data['attention_mask1'].to(device)
      input_ids2 = data['input_ids2'].to(device)
      attention_mask2 = data['attention_mask2'].to(device)
      labels = data["label"].to(device)
      outputs = model(
        input_ids1=input_ids1,
        attention_mask1=attention_mask1,
        input_ids2 = input_ids2,
        attention_mask2 = attention_mask2
      )
      #_, preds = torch.max(outputs, dim=1)
      preds = [0 if x < 0.5 else 1 for x in outputs]
      preds = torch.tensor(preds).to(device)      
      sentence.extend(texts)
      predictions.extend(preds)
      prediction_probs.extend(outputs)
      real_values.extend(labels)
  predictions = torch.stack(predictions).cpu()
  prediction_probs = torch.stack(prediction_probs).cpu()
  real_values = torch.stack(real_values).cpu()
  return sentence, predictions, prediction_probs, real_values

In [20]:
y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(model,test_data_loader)

In [24]:
from sklearn.metrics import classification_report,confusion_matrix,f1_score
class_name = ['Hope_speech','Non_hope_speech']
print(classification_report(y_test, y_pred, target_names=class_name,zero_division=0, digits=3))

                 precision    recall  f1-score   support

    Hope_speech      0.690     0.614     0.650       228
Non_hope_speech      0.788     0.838     0.812       390

       accuracy                          0.756       618
      macro avg      0.739     0.726     0.731       618
   weighted avg      0.752     0.756     0.752       618

